In [1]:
!pip install farm-haystack

In [2]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)
data_dir = "./"
# data_dir = "PATH/TO_YOUR/TRAIN_DATA"


C:\Users\kapil\anaconda3\envs\mlai1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#reader.train(data_dir=data_dir, train_filename="answers_gst.json", use_gpu=False, n_epochs=10, save_dir="gst_10",)
new_reader = FARMReader(model_name_or_path="gst_10")

C:\Users\kapil\anaconda3\envs\mlai1\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": true, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}


In [5]:
context = '''DATTATRAYA MAHARAJ KALAMBE JALOLI SAHAKARI BANK LTD. SECOND FLOOR, DMK JAOLI BANK BUILDING, HO, 418/20, MAULANA AZAD ROAD, NEAR ROUND TEMPLE, MU MBAI INVOICE DATE: 31-12-2022 CUSTOMER:ICICI BANK LTD. ADDRESS :UNIT NO 503 5TH FLOOR HALLMARKBUSINESS PLAZA SANT DNYANESHWAR MARG BANDRA E INVOICE NO: AST MUMBAI MAHARASHTRA 400051 STATE :MAHARASHTRA DMKBUP|112220045 STATE CODE :27 GSTIN :27AAACI1195HSZI SR.NO. DESCRIPTION HSN CODE DETAILS COUNT UNIT AMOUNT(RS.) INTERCHANGE RECEIVED 1 FOR THE MONTH OF DEC-22 997158 FUND TRANSFER TRANSACTIONS 15 55.30 MERCHANT TRANSACTIONS TOTAL 55.30 CGST @ 9% 4.98 SGST/UGST @9%% 4.98 IGST @ 18% 0,00 GRAND TOTAL 65.26 TOTAL (AMOUNT IN WORDS) RUPEES SIXTY FIVE AND TWENTY SIX PAISE ONLY FOR DATTATRAYA MAHARAJ KALAMBE JALOLI SAHAKARI BANK LTD. GSTIN: 27AAAAD4765G1ZR PAN: AAAAD4765G STATE: MAHARASHTRA APPROVAL SIGNATURE SIGNOR: ARVIND SURVE DATA: 28 MARCH 2023 18:34 REASON: DIGITALLY SIGNED'''

In [4]:
from haystack import Pipeline, Document
from haystack.utils import print_answers
# reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
p = Pipeline()
p.add_node(component=new_reader, name="Reader", inputs=["Query"])

def find_lables(context):
    
    output = {}
    lables = ["icici gst number? ","pdf address?","pdf bank name?","pdf cgst @9?","pdf igst @18?",\
             "PDF_GSTIN	","PDF_Interchange Fees	","PDF_Invoice date	","PDF_Invoice Number",\
              "PDF_SGST / UGST @ 9 %", "Signature Remarks	"]
    for lbl in lables:
        res = p.run(
            query=lbl, documents=[Document(content=context)]
        )
        out = [x.to_dict() for x in res["answers"]]
       # print(out)
     #   print("Query -> {} : Answer is -> {} with {}".format(lbl,out[0]['answer'],out[0]['score']))

        output[lbl] =  str(out[0]['answer']) + " ==> " +str(out[0]['score'])
    return output



In [5]:
from PyPDF2 import PdfReader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from pprint import pprint

from doctr.io import DocumentFile
from doctr.models import ocr_predictor
model_ocr = ocr_predictor(det_arch='db_resnet50', reco_arch='crnn_vgg16_bn', pretrained=True)
import itertools
import warnings
import gradio as gr
warnings.filterwarnings("ignore")

import cv2,requests,time,json,glob
import numpy as np
from pathlib import Path    

import fitz
from tqdm import tqdm

subscription_key = "cd9de642c81c46ddbe6509ad84c9b621"#"8b77eadc000640d4a292194794995dc6"#"94e072db7b7f451f82252fa11686c1e5" #"cd9de642c81c46ddbe6509ad84c9b621"
text_recognition_url = "https://centralindia.api.cognitive.microsoft.com/vision/v2.0/read/core/asyncBatchAnalyze"

Path("images").mkdir(parents=True, exist_ok=True)

def find_ocr_ms(filename):

  doc = fitz.open(filename)
  
  images = []

  try:
   for page in tqdm(doc):


      pm = page.get_pixmap(matrix=fitz.Identity, alpha=False)
   #   print("images/samplepdfimage-%i.jpg" % page.number)
      images.append("images/samplepdfimage-%i.jpg" % page.number)
      pm.save("images/samplepdfimage-%i.jpg" % page.number)  # save file
  except:
    pass

  total_ocr = []
  for i,filename in enumerate(images):
          image_url = cv2.imread(filename)           # GIVE IMAGE PATH HERE
          out_path = Path(filename).name
        #  print(out_path)

          
          success, encoded_image = cv2.imencode('.jpg', image_url)
          image_data = encoded_image.tobytes()    								
          headers = {'Ocp-Apim-Subscription-Key': subscription_key,
                'Content-Type': 'application/octet-stream'}
          params = {'visualFeatures': 'Categories,Description,Color','language': 'unk'}
          response = requests.post(text_recognition_url, headers=headers, params=params, data=image_data)
          response.raise_for_status()
       
          operation_url = response.headers["Operation-Location"]
          
          analysis = {}
          poll = True
          raw = []
          while (poll):
              response_final = requests.get(operation_url, headers=headers)
              analysis = response_final.json()
              time.sleep(0.1)
              if ("recognitionResults" in analysis):
                  poll= False 
              if ("status" in analysis and analysis['status'] == 'Failed'):
                  poll= False
          
          polygons=[]
          if ("recognitionResults" in analysis):
              polygons = [(line["boundingBox"], line["text"])
                          for line in analysis["recognitionResults"][0]["lines"]]
          
          
          words = []
          for polygon in polygons:
              vertices = [(polygon[0][i], polygon[0][i+1])
                          for i in range(0, len(polygon[0]), 2)]
              start_pt,end_pt = vertices[0], vertices[2]
              text     = polygon[1].upper()
              words.append((text))
  return " ".join(words)             
 # my_path = Path('images')
 # my_path.rmtree(ignore_errors=True)


def find_ocr(file):
    print(file.name)
    doc = DocumentFile.from_pdf(str(file.name))

    # Analyze
    result = model_ocr(doc)

#    result = model(doc[5:6])
    json_output = result.export()
    words_loc_normalized = []
    corpus = []
    for ii in range (len(json_output['pages'])):
      for d in json_output.values():
          num_blocks = len(d[ii]['blocks'])
      #    print(num_blocks)
  
          for k in range (num_blocks):
          
            row = d[ii]['blocks'][k]['lines']
      #      print(row)
            res = " ".join([k['value'] for item in row for k in item['words']])
         #   print(res)
            corpus.append(res)
            
    return " ".join(corpus)

if __name__ == "__main__":
     
 
    with gr.Blocks() as demo:
    
         file = gr.File()
         docs = gr.Radio(["corporate_gurantee","gst_invoices"], label="Doc_type")
         print(docs)
    
         output = gr.Textbox()
         btn = gr.Button(value="Get OCR")
         btn.click(find_ocr_ms, inputs=[file], outputs=[output])
         
         with gr.Row():
             output2 = gr.JSON()
    
         btn = gr.Button(value="Extract info")
         btn.click(find_lables, inputs=[output], outputs=[output2])
         
    demo.launch()

C:\Users\kapil\anaconda3\envs\mlai1\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\kapil\anaconda3\envs\mlai1\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


radio
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.34s/ Batches]
